In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
lakshmi25npathi_imdb_dataset_of_50k_movie_reviews_path = kagglehub.dataset_download('lakshmi25npathi/imdb-dataset-of-50k-movie-reviews')

print('Data source import complete.')


Data source import complete.


In [3]:
!pip install spacy

!python -m spacy download en_core_web_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report
import spacy

In [5]:
df=pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
df.isna().sum()

review       0
sentiment    0
dtype: int64

In [7]:
# check the distribution
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [8]:
# show sample
print(f"{df['review'][0]} -> {df['sentiment'][0]}")

One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fac

In [9]:
#load eng language
nlp=spacy.load("en_core_web_sm")

In [10]:
txt=df['review'][5]
txt

'Probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it\'s not preachy or boring. It just never gets old, despite my having seen it some 15 or more times in the last 25 years. Paul Lukas\' performance brings tears to my eyes, and Bette Davis, in one of her very few truly sympathetic roles, is a delight. The kids are, as grandma says, more like "dressed-up midgets" than children, but that only makes them more fun to watch. And the mother\'s slow awakening to what\'s happening in the world and under her own roof is believable and startling. If I had a dozen thumbs, they\'d all be "up" for this movie.'

In [11]:
doc=nlp(txt)
doc

Probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it's not preachy or boring. It just never gets old, despite my having seen it some 15 or more times in the last 25 years. Paul Lukas' performance brings tears to my eyes, and Bette Davis, in one of her very few truly sympathetic roles, is a delight. The kids are, as grandma says, more like "dressed-up midgets" than children, but that only makes them more fun to watch. And the mother's slow awakening to what's happening in the world and under her own roof is believable and startling. If I had a dozen thumbs, they'd all be "up" for this movie.

In [12]:

for sentence in doc.sents:
     print(sentence)

Probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it's not preachy or boring.
It just never gets old, despite my having seen it some 15 or more times in the last 25 years.
Paul Lukas' performance brings tears to my eyes, and Bette Davis, in one of her very few truly sympathetic roles, is a delight.
The kids are, as grandma says, more like "dressed-up midgets" than children, but that only makes them more fun to watch.
And the mother's slow awakening to what's happening in the world and under her own roof is believable and startling.
If I had a dozen thumbs, they'd all be "up" for this movie.


In [13]:
for token in doc:
    print(token)

Probably
my
all
-
time
favorite
movie
,
a
story
of
selflessness
,
sacrifice
and
dedication
to
a
noble
cause
,
but
it
's
not
preachy
or
boring
.
It
just
never
gets
old
,
despite
my
having
seen
it
some
15
or
more
times
in
the
last
25
years
.
Paul
Lukas
'
performance
brings
tears
to
my
eyes
,
and
Bette
Davis
,
in
one
of
her
very
few
truly
sympathetic
roles
,
is
a
delight
.
The
kids
are
,
as
grandma
says
,
more
like
"
dressed
-
up
midgets
"
than
children
,
but
that
only
makes
them
more
fun
to
watch
.
And
the
mother
's
slow
awakening
to
what
's
happening
in
the
world
and
under
her
own
roof
is
believable
and
startling
.
If
I
had
a
dozen
thumbs
,
they
'd
all
be
"
up
"
for
this
movie
.


In [14]:
for token in doc:
    print(f"Word: {token} | -> {token.lemma_}")

Word: Probably | -> probably
Word: my | -> my
Word: all | -> all
Word: - | -> -
Word: time | -> time
Word: favorite | -> favorite
Word: movie | -> movie
Word: , | -> ,
Word: a | -> a
Word: story | -> story
Word: of | -> of
Word: selflessness | -> selflessness
Word: , | -> ,
Word: sacrifice | -> sacrifice
Word: and | -> and
Word: dedication | -> dedication
Word: to | -> to
Word: a | -> a
Word: noble | -> noble
Word: cause | -> cause
Word: , | -> ,
Word: but | -> but
Word: it | -> it
Word: 's | -> be
Word: not | -> not
Word: preachy | -> preachy
Word: or | -> or
Word: boring | -> boring
Word: . | -> .
Word: It | -> it
Word: just | -> just
Word: never | -> never
Word: gets | -> get
Word: old | -> old
Word: , | -> ,
Word: despite | -> despite
Word: my | -> my
Word: having | -> having
Word: seen | -> see
Word: it | -> it
Word: some | -> some
Word: 15 | -> 15
Word: or | -> or
Word: more | -> more
Word: times | -> time
Word: in | -> in
Word: the | -> the
Word: last | -> last
Word: 25 | -> 25


In [15]:
for token in doc:
    if token.is_stop or token.is_punct:
        print(token)

my
all
-
,
a
of
,
and
to
a
,
but
it
's
not
or
.
It
just
never
,
my
it
some
or
more
in
the
last
.
'
to
my
,
and
,
in
one
of
her
very
few
,
is
a
.
The
are
,
as
,
more
"
-
up
"
than
,
but
that
only
them
more
to
.
And
the
's
to
what
's
in
the
and
under
her
own
is
and
.
If
I
had
a
,
they
'd
all
be
"
up
"
for
this
.


In [16]:
# use this utility function to get the preprocessed text data
def preprocess(text):
    # remove stop words and lemmatize the text
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)

    return " ".join(filtered_tokens)

In [17]:
print(txt)
print('******************************************************')
procces_txt = preprocess(txt)
print(procces_txt)

Probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it's not preachy or boring. It just never gets old, despite my having seen it some 15 or more times in the last 25 years. Paul Lukas' performance brings tears to my eyes, and Bette Davis, in one of her very few truly sympathetic roles, is a delight. The kids are, as grandma says, more like "dressed-up midgets" than children, but that only makes them more fun to watch. And the mother's slow awakening to what's happening in the world and under her own roof is believable and startling. If I had a dozen thumbs, they'd all be "up" for this movie.
******************************************************
probably time favorite movie story selflessness sacrifice dedication noble cause preachy boring get old despite having see 15 time 25 year Paul Lukas performance bring tear eye Bette Davis truly sympathetic role delight kid grandma say like dress midget child make fun watch mother slow a

In [18]:
df['preprocessed_review'] = df['review'].apply(preprocess)

In [19]:
df.head()

,review,sentiment,preprocessed_review
0,One of the other reviewers has mentioned that ...,positive,reviewer mention watch 1 oz episode hook right...
1,A wonderful little production. <br /><br />The...,positive,wonderful little production < br /><br />the f...
2,I thought this was a wonderful way to spend ti...,positive,think wonderful way spend time hot summer week...
3,Basically there's a family where a little boy ...,negative,basically family little boy Jake think zombie ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,Petter Mattei love Time money visually stunnin...


In [20]:
df['sentiment_num'] = df['sentiment'].map({'positive' : 1, 'negative': 0})

df.head(5)

,review,sentiment,preprocessed_review,sentiment_num
0,One of the other reviewers has mentioned that ...,positive,reviewer mention watch 1 oz episode hook right...,1
1,A wonderful little production. <br /><br />The...,positive,wonderful little production < br /><br />the f...,1
2,I thought this was a wonderful way to spend ti...,positive,think wonderful way spend time hot summer week...,1
3,Basically there's a family where a little boy ...,negative,basically family little boy Jake think zombie ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,Petter Mattei love Time money visually stunnin...,1


In [21]:
x=df['preprocessed_review']
y=df['sentiment_num']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [22]:
v = TfidfVectorizer()

X_train_cv = v.fit_transform(X_train)
X_test_cv = v.transform(X_test)

# All TfidfVectorizer vocabularies
print(v.vocabulary_)

{'keep': 39823, 'ask': 5115, 'fight': 26670, 'scream': 64299, 'match': 45797, 'swearing': 71332, 'general': 29569, 'mayhem': 46038, 'permeate': 54925, '84': 1138, 'minute': 47765, 'comparison': 15350, 'stand': 69132, 'think': 73189, 'dimensional': 20325, 'character': 13082, 'little': 43151, 'depth': 19528, 'virtually': 78913, 'impossible': 36098, 'care': 12003, 'happen': 32683, 'badly': 6219, 'write': 81543, 'cypher': 17919, 'director': 20471, 'hang': 32607, 'multicultural': 49321, 'belief': 7543, 'topic': 74115, 'well': 80111, 'drama': 21856, 'tv': 75521, 'cinema': 14055, 'br': 9836, 'confess': 15683, 'spot': 68812, 'bad': 6181, 'performance': 54859, 'film': 26729, 'say': 63685, 'nichola': 50706, 'burley': 11010, 'heroine': 33744, 'slutty': 67290, 'good': 30712, 'friend': 28389, 'wasim': 79735, 'zakir': 82337, 'nasty': 50052, 'bully': 10870, 'brother': 10490, 'absolutely': 1498, 'terrible': 72775, 'know': 40645, 'act': 1791, 'school': 64047, 'graduate': 31030, 'apply': 4400, 'refund':

In [23]:
RFC = RandomForestClassifier()

RFC.fit(X_train_cv, y_train)

RandomForestClassifier()

In [24]:
y_pred = RFC.predict(X_test_cv)

In [25]:
print(accuracy_score(y_test, y_pred))

0.8538


In [26]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.86      0.85      4961
           1       0.86      0.85      0.85      5039

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



In [27]:
test_text = df['review'][2000]
test_text

"Stranded in Space (1972) MST3K version - a very not good TV movie pilot, for a never to be made series, in which an astronaut finds himself trapped on Earth's evil twin. Having a planet of identical size and mass orbiting in the same plane as the earth, but on the opposite side of the sun, is a well worn SF chestnut - the idea is over 2,000 years old, having been invented by the Ancient Greeks. In this version the Counter World is run as an Orwellian 'perfect' society. Where, for totally inexplicable reasons, everyone speaks English and drives late model American cars. After escaping from his prisonlike hospital, the disruptive Earthian is chased around Not Southern California by TV and bad movie stalwart Cameron Mitchell who, like his minions, wears double breasted suits and black polo neck jumpers - a stylishly evil combination which I fully intend to adopt if ever I become a totalitarian overlord. Our hero escapes several times before ending up gazing at the alien world's three moo

In [28]:
# Apply preprocess
test_text_processed = [preprocess(test_text)]
test_text_processed

['strand Space 1972 MST3 K version good tv movie pilot series astronaut find trap Earth evil twin have planet identical size mass orbit plane earth opposite sun worn SF chestnut idea 2,000 year old having invent Ancient Greeks version Counter World run Orwellian perfect society totally inexplicable reason speak English drive late model american car escape prisonlike hospital disruptive Earthian chase southern California tv bad movie stalwart Cameron Mitchell like minion wear double breasted suit black polo neck jumper stylishly evil combination fully intend adopt totalitarian overlord hero escape time end gaze alien world moon wonder aloud home set Man Hostile World make new friend Week move end Episode show beloved industry 70 80 Fugitive Incredible Hulk Littlest Hobo etc curiously weird bit title sequence strand Space air title Stranger 1972 MST3 K version 1991 acquire footage 1983 movie Prisoners Lost Universe 1991 opening credit strand Space run shot people fall matter transmitter 

In [29]:
test_text_vc = v.transform(test_text_processed)

In [30]:
test_text = RFC.predict(test_text_vc)

In [31]:
print(f"{df['review'][2000]} -> {df['sentiment_num'][2000]}")
print(test_text)

Stranded in Space (1972) MST3K version - a very not good TV movie pilot, for a never to be made series, in which an astronaut finds himself trapped on Earth's evil twin. Having a planet of identical size and mass orbiting in the same plane as the earth, but on the opposite side of the sun, is a well worn SF chestnut - the idea is over 2,000 years old, having been invented by the Ancient Greeks. In this version the Counter World is run as an Orwellian 'perfect' society. Where, for totally inexplicable reasons, everyone speaks English and drives late model American cars. After escaping from his prisonlike hospital, the disruptive Earthian is chased around Not Southern California by TV and bad movie stalwart Cameron Mitchell who, like his minions, wears double breasted suits and black polo neck jumpers - a stylishly evil combination which I fully intend to adopt if ever I become a totalitarian overlord. Our hero escapes several times before ending up gazing at the alien world's three moon